**Import packages**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import torch
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig
import torch.nn as nn


**Load the dataset**

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self):
        pass


In [3]:
dataset_dir = './dataset/NYT29/'

file_types = ['.sent', '.tup']
datasets = ['train', 'test', 'dev']

train = {}
test = {}
dev = {}

for d in datasets:
    for t in file_types:
        with open(os.path.join(dataset_dir, f'{d}{t}'), 'r') as f:
            if t == '.sent':
                exec(f'{d}["sent"] = f.readlines()')
            else:
                exec(f'{d}["tup"] = f.readlines()')

In [4]:
MODEL_NAME = 'bert-base-uncased'

{'sent': ['then terrorism struck again , this time in the indonesia capital of jakarta .\n',
  'a12 new york\\/region b1-7 enclave for middle class is put up for sale the owner of stuyvesant town and peter cooper village , two 60-year-old apartment complexes on the east side of manhattan , is auctioning them for a target price of nearly $ 5 billion .\n',
  "before long , though , he 's continent-hopping , working as a deckhand on a trawler in alaska ; chasing gold in nicaragua ; dodging bullets in pamplona , spain , where he and a friend stumble into some basque riots . ''\n",
  "general casey said the iraqi forces had little control of the country 's border with syria on either side of qaim , a desolate town on the euphrates .\n",
  '84 , of avon , connecticut and longboat key , florida , died friday , june 10 , at home .\n',
  "since a midterm election victory in october , mr. kirchner has also moved to establish an alliance with venezuela 's populist leader , hugo chávez , and , as 